In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)

In [4]:
import os
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict
import re
import spacy
from spacy import displacy
#import gensim
#import nltk


In [3]:
test_aspect_pred = pd.read_csv('../output/BiLSTM_aspect_prediction_test_data.csv')

In [4]:
import ast 
test_aspect_pred['aspects_pred'] = test_aspect_pred['aspects_pred'].apply(lambda x: ast.literal_eval(x))
test_aspect_pred['aspects'] = test_aspect_pred['aspects'].apply(lambda x: ast.literal_eval(x))

In [5]:
test_aspect_pred['ct_aspects'] = test_aspect_pred['aspects_pred'].apply(lambda x: len(x))
test_aspect_pred['ct_aspects'].value_counts()

1    465
2    142
3    26 
0    15 
4    1  
Name: ct_aspects, dtype: int64

In [6]:
test_aspect_pred[test_aspect_pred['ct_aspects']==0].tail()

,text,ind,aspects_pred,aspects,ct_aspects
447,so garantuan that you can't fit them in a toaster.,0,[],[food],0
464,The sandwhiches are out-of-this world!,0,[],[food],0
492,So eat fast and order a lot.,0,[],[anecdotes/miscellaneous],0
584,We were a group of 8 and well seved.,1,[],"[anecdotes/miscellaneous, service]",0
632,The restaurant is rather small but we were lucky to get a table quickly.,1,[],"[ambience, service]",0


##### We will try to break the review text using constituency parsing . If the reviews contain constrating conjuction and they have not been split by constituency parsing method , we will try to split them using constrasting conjuction.

In [7]:
import time
import benepar

def traverse_tree(sentence):
    res = []
    parser = benepar.Parser("benepar_en2")
    tree = parser.parse(sentence)
    for subtree in tree:
        if type(subtree) == nltk.tree.Tree:
            type_list = ['S' ] 
            if  subtree.label() in type_list :            
                res.append(subtree.leaves())
    if len(res)==0:
        res = [sentence.split(' ')]
    for ele in res:
        if isinstance(ele , list)  and len(ele)==1:
            res.remove(ele)
    return res

In [8]:
import time
st = time.time()
test_aspect_pred['splits'] = test_aspect_pred.apply(lambda x: traverse_tree(x['text']) if x['ct_aspects'] >1 else [x['text'].split(' ')] , axis = 1 )
en = time.time()
print('time taken in minutes',(en -st)/60)

time taken in minutes 48.015214228630064


If reading the saved csv file, need to convert splits column again using ast literal eval 

In [11]:
#test_aspect_pred['splits']= test_aspect_pred['splits'].apply(lambda x: ast.literal_eval(x))
test_aspect_pred['length_split'] = test_aspect_pred['splits'].apply(lambda x: len(x))
test_aspect_pred.length_split.value_counts()

1    598
2    45 
3    5  
4    1  
Name: length_split, dtype: int64

In [21]:
test_aspect_pred.tail()

,text,ind,aspects_pred,aspects,ct_aspects,splits
644,We were fans of the half-price Saturday night option until some inedible squid during a recent visit.,1,[price],"[food, price]",1,"[[We, were, fans, of, the, half-price, Saturday, night, option, until, some, inedible, squid, during, a, recent, visit.]]"
645,"You don't go to Mizu for excellent service, you go for the large amounts of food, the amiable atmosphere, and the hole-in-the-wall feeling of the place.",1,"[ambience, food, service]","[ambience, food, service]",3,"[[You, do, n't, go, to, Mizu, for, excellent, service], [you, go, for, the, large, amounts, of, food, ,, the, amiable, atmosphere, ,, and, the, hole-in-the-wall, feeling, of, the, place]]"
646,"They might be all business at the counter when you give your order, but their food says I love you.",1,[service],"[food, service]",1,"[[They, might, be, all, business, at, the, counter, when, you, give, your, order,, but, their, food, says, I, love, you.]]"
647,"I would recommend Roxy's for that, but not for their food.",1,[food],"[anecdotes/miscellaneous, food]",1,"[[I, would, recommend, Roxy's, for, that,, but, not, for, their, food.]]"
648,"I went here with a friend on a whim, we went someplace else first and couldn't get a table.",1,[anecdotes/miscellaneous],"[anecdotes/miscellaneous, service]",1,"[[I, went, here, with, a, friend, on, a, whim,, we, went, someplace, else, first, and, couldn't, get, a, table.]]"


In [9]:
import ast
#test_aspect_pred.to_csv('../output/test_data_sentence_segmentation.csv' , index = False)
test_aspect_pred = pd.read_csv('../output/test_data_sentence_segmentation.csv')
for col in ['aspects_pred' , 'aspects' , 'splits']:
    test_aspect_pred[col]  = test_aspect_pred[col].apply(lambda x: ast.literal_eval(x))


In [11]:
test_aspect_pred.tail(2)

,text,ind,aspects_pred,aspects,ct_aspects,splits
647,"I would recommend Roxy's for that, but not for their food.",1,[food],"[anecdotes/miscellaneous, food]",1,"[[I, would, recommend, Roxy's, for, that,, but, not, for, their, food.]]"
648,"I went here with a friend on a whim, we went someplace else first and couldn't get a table.",1,[anecdotes/miscellaneous],"[anecdotes/miscellaneous, service]",1,"[[I, went, here, with, a, friend, on, a, whim,, we, went, someplace, else, first, and, couldn't, get, a, table.]]"


In [12]:
test_aspect_pred['length_splits'] = test_aspect_pred['splits'].apply(lambda x: len(x))
test_aspect_pred.length_splits.value_counts()

1    598
2    45 
3    5  
4    1  
Name: length_splits, dtype: int64

In [13]:
single_aspect_df = test_aspect_pred[(test_aspect_pred['length_splits']==1)] # & (test_aspect_pred['ct_aspects']<=1)
multi_aspect_df = test_aspect_pred[test_aspect_pred['length_splits']>1]

In [14]:
multi_aspect_df.drop(columns=['aspects' , 'ct_aspects', 'length_splits' ] , inplace=True)

/home/FRACTAL/swati.tiwari/anaconda3/envs/fastai38/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [15]:
text_aspectwise= []
for index, row in multi_aspect_df.iterrows():                   
    total_aspects = row['aspects_pred'].copy()
    total_sent = row['splits'].copy()    
    for sent in total_sent:
        text_aspectwise.append({  'id': index , 'sp_text': ' '.join(sent) , 'aspects_pred': total_aspects })

df_test_aspectwise = pd.DataFrame(text_aspectwise)

In [16]:
df_test_aspectwise.shape

(109, 3)

In [19]:
import pickle 
new_vocab = pickle.load(open('../../../data/manually_tweaked_dict.pickle' , 'rb'))
new_vocab['price'].append('pricy')
new_vocab['price'].append('dollar')

In [22]:
len(new_vocab['price'])

17

In [24]:
new_vocab.keys()

dict_keys(['ambience', 'food', 'price', 'service'])

In [ ]:



        for aspect , polarity in zip(total_aspects ,total_polarities):
            match_words = []
            for sen_list in row['conj_splits']:
                if aspect in sen_list:
                    match_words.append(999)  ## Exact word present in the sentence
                else:
                    words = sum(word.lower() in new_vocab[aspect] for word in sen_list)
                    match_words.append(words)

In [18]:
df_test_aspectwise

,aspects_pred,id,sp_text
0,"[food, service]",3,We were seated outside
1,"[food, service]",3,the waiter spilled red wine and hot tea on myself and my date
2,"[food, service]",4,The crust is thin
3,"[food, service]",4,the ingredients are fresh
4,"[food, service]",4,the staff is friendly
5,"[ambience, food, service]",14,We were still sitting at the bar while we drank the sangria
6,"[ambience, food, service]",14,"facing away from the bar when we turned back around , the dollar 2 was gone the people next to us said the bartender took it"
7,"[food, price]",75,The drinks are always welll made
8,"[food, price]",75,wine selection is fairly priced
9,"[anecdotes/miscellaneous, food]",88,I 've been to several places for Dim Sum


In [ ]:
test_preds = []
text_data = []
true_label = []

epoch_loss = 0
epoch_acc = 0 
ct = 0
model.eval()
with torch.no_grad():
    for x ,y  in test_loader:
        text, text_lengths = x
        text_data.append(text.data.cpu().numpy())
        predictions = model(text, text_lengths)#.squeeze(1)
        rounded_preds = torch.round(torch.sigmoid(predictions))  #torch.round
        preds = rounded_preds.data.cpu().numpy()
        test_preds.append(preds)
        true_label.append(y.data.cpu().numpy())


In [11]:
import pickle 
new_vocab = pickle.load(open('../../data/manually_tweaked_dict.pickle' , 'rb'))
new_vocab['price'].append('pricy')
new_vocab['price'].append('dollar')

In [12]:
"""
some of the lists  are empty/ or contains ' ' characters only. Removing them using following function
"""
def remove_null(lists):
    for lst in lists:
        if isinstance(lst , list) & ( lst ==[''] or lst==[' '] ) :
            lists.remove(lst)
    return lists 

In [13]:
test_grp_df['conj_splits'] = test_grp_df['conj_splits'].apply(lambda x: remove_null(x))
test_grp_df['length_split'] = test_grp_df['conj_splits'].apply(lambda x: len(x))
test_grp_df.length_split.value_counts()

1    464
2    165
3    13 
4    1  
Name: length_split, dtype: int64

In [14]:
test_grp_df.reset_index(inplace=True , drop = True)
test_grp_df['aspects'] = test_grp_df['aspects'].apply(lambda x: [i for i in x.split(' ')])
test_grp_df['polarities'] = test_grp_df['polarities'].apply(lambda x: [i for i in x.split(' ')])

Once we have sentence fragments, we need to map each sentence fragments to corresponding aspect categories. Created a dictionary containing key words corresponding to each aspect categories. And implementing following steps: 

1) if only one fragment in splits , assign complete sentence to each of the aspect categories. 

2) For each aspect category, look at each fragment  and count # of keywords present. Assign fragment with higher matching keywords to given aspect.

3) if we have zero matching keyword in all the sentence fragments, assign the complete sentenence to that aspect category.

In [16]:
text_aspectwise= []
for index , row in test_grp_df.iterrows():

    if row['length_split']==1:
        for aspect , polarity in zip(row['aspects'] , row['polarities']):
            text_aspectwise.append({'id':index ,'text':row['text'].lower(), 'aspect': aspect,'polarity': polarity})
            
    else:                   
        total_aspects = row['aspects'].copy();   total_polarities = row['polarities'].copy()
        for aspect , polarity in zip(total_aspects , total_polarities):
            if aspect == 'anecdotes/miscellaneous':
                text_aspectwise.append({'id':index ,'text':row['text'].lower(), 'aspect': aspect, 'polarity': polarity})
                total_aspects.remove(aspect); total_polarities.remove(polarity)
                break

        total_sent = row['conj_splits'].copy()
        for aspect , polarity in zip(total_aspects ,total_polarities):
            match_words = []
            for sen_list in row['conj_splits']:
                if aspect in sen_list:
                    match_words.append(999)  ## Exact word present in the sentence
                else:
                    words = sum(word.lower() in new_vocab[aspect] for word in sen_list)
                    match_words.append(words)
            if np.max(match_words)!=0:
                best_sent = row['conj_splits'][np.argmax(match_words)]
                text_aspectwise.append({'id':index,'text':' '.join(best_sent),   'aspect': aspect,'polarity': polarity})
            else:
                
                text_aspectwise.append({'id':index,'text':row['text'],   'aspect': aspect,'polarity': polarity})
                


In [17]:
df_aspectwise_test =    pd.DataFrame.from_records(text_aspectwise)
df_aspectwise_test.to_csv('../../data/test_data_aspectXtext_fragment.csv')